<head><h1 align="center">
Food.com Recipe Interactions
</h1></head>  
  
<head><h3 align="center">Recipe Recommender System Modeling</h3></head>

We will develop a **personalized recommender system**. With such systems, there are different options for methods in which to approach recommendations:
  
- The first method is to ask for *explicit* defined ratings from a user regarding the content that they have used/viewed/consumed.  

- Another method is to gather data *implicitly* as the user interacts with the system or service.
  
In this dataset, we have both:  
- **Explicit Data** — the `rating` feature of the **interactions dataset** for each `user`.
- **Implicit Data** — the `x` feature of the **x dataset**.  
  
Technically, I would suppose the `review` feature would fall under explicit data as well, but to use it as such would require *Natural Language Processing*, which we may get involved in later, but not in this notebook.  
  
Additionally with personalized recommender systems, we can center our system around either around suggesting similar *content* (recipes) or suggestions based on other *users of similar preference*. Both systems are succinctly described below:
  
**Content-Based Recommenders**  
> Main Idea: If you like an item, you will also like "similar" items.$_1$  
  
**Collaborative Filtering Systems**  
> Main Idea: If user A likes items 5, 6, 7, and 8 and user B likes items 5, 6, and 7, then it is highly likely that user B will also like item 8.$_1$  
  


#### Setup

In [105]:
import re
import os
import time
import csv
import json
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from os import system
from math import floor
from copy import deepcopy
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 50)

In [117]:
from surprise.model_selection import train_test_split, GridSearchCV
from surprise.prediction_algorithms import knns, SVD
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy

In [124]:
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV

In [125]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist

**Data Preparation**

In [120]:
# Read in data
df = pd.read_csv('data/RAW_recipes.csv')
int_df = pd.read_csv('data/RAW_interactions.csv')
tdf = pd.read_csv('data/interactions_train.csv')
vdf = pd.read_csv('data/interactions_validation.csv')

# """Cleaning: All steps carried over from EDA Notebook"""
df.drop(labels=721, inplace = True)

# Cleaning/FE: Creating columns for recipe's respective nutrients
df['kcal'] = df.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[0])
df['fat'] = df.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[1])
df['sugar'] = df.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[2])
df['salt'] = df.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[3])
df['protein'] = df.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[4])
df['sat_fat'] = df.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[5])
df['carbs'] = df.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[6])

# Cleaning: Imputing outlier value to median
df['minutes'] = np.where(df.minutes == 2147483647,
                         df.minutes.median(),
                         df.minutes)

# Setting these two dataframes to independent .head() call sizes for ease of access
idf = int_df.head(11)
rdf = df.head(2)

**Functions**

In [112]:
idf

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."
5,52282,120345,2005-05-21,4,very very sweet. after i waited the 2 days i b...
6,124416,120345,2011-08-06,0,"Just an observation, so I will not rate. I fo..."
7,2000192946,120345,2015-05-10,2,This recipe was OVERLY too sweet. I would sta...
8,76535,134728,2005-09-02,4,Very good!
9,273745,134728,2005-12-22,5,Better than the real!!


In [113]:
rdf

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,kcal,fat,sugar,salt,protein,sat_fat,carbs
0,arriba baked winter squash mexican style,137739,55.0,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,31490,30.0,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0


# Collaborative Filtering — User-based Recommender System

>- Collaborative Filtering (CF) is currently the most widely used approach to build recommendation systems  
>- The key idea behind CF is that similar users have similar interests and that a user generally likes items that are similar to other items they like  
>- CF is filling an "empty cell" in the utility matrix based on the similarity between users or item. Matrix factorization or decomposition can help us solve this problem by determining what the overall "topics" are when a matrix is factored
  
Note: We will likely need to bring this notebook over to DataBricks or AWS.  
  
We can use **cosine similarity** between users, or **Pearson Correlation Coefficient**. Different metrics will offer different results, but I don't see why we couldn't offer multiple recommendations using different metrics.

**Note: Don't forget to downplay zeros as they are gaps, not values representative of sentiment!**

### SVD — Singular Value (Matrix) Decomposition

Notes:  
- eigendecompositions require square matrices
- for SVD, we can create square matrices by taking the dot product of a matrix and its transpose
- we can use SVD to draw vectors in a new space to capture as much of the variance in our data as possible

In [126]:
idf.columns

Index(['user_id', 'recipe_id', 'date', 'rating', 'review'], dtype='object')

~Below we use the raw interactions dataset to create the rating matrix `A`, with rows representing recipes and columns representing users.~  
NVM LOOKS LIKE WE'RE GONNA NEED DISTRIBUTED COMPUTING FOR EVEN A SIMPLE SVD


In [127]:
A = np.ndarray(
    shape=(np.max(idf.recipe_id.values), np.max(idf.user_id.values)),
    dtype=np.uint8)

A[idf.recipe_id.values-1, idf.user_id.values-1] = idf.rating.values

MemoryError: Unable to allocate 245. TiB for an array with shape (134728, 2000192946) and data type uint8

Let's try with our smaller prepackaged training set—`tdf`

In [128]:
A = np.ndarray(
    shape=(np.max(tdf.recipe_id.values), np.max(tdf.user_id.values)),
    dtype=np.uint8)

A[tdf.recipe_id.values-1, tdf.user_id.values-1] = tdf.rating.values

MemoryError: Unable to allocate 979. TiB for an array with shape (537458, 2002312797) and data type uint8

$_{my}$ $_{poor}$ $_{memories.}$ $_{they're}$ $_{that}$ $_{I}$ $_{all}$ $_{I}$ $_{have}$ ._.'

### ALS — Alternating Lease Squares Matrix Decomposition

- plug in guess values for P and Q
- hold the values of one constant, then use the values for R and the non-constant to find the optimum values for.
- repeat for the other

> "When we talk about collaborative filtering for recommender systems we want to solve the problem of our original matrix having millions of different dimensions, but our 'tastes' not being nearly as complex. Even if i’ve \[sic\] viewed hundreds of items they might just express a couple of different tastes. Here we can actually use matrix factorization to mathematically reduce the dimensionality of our original 'all users by all items' matrix into something much smaller that represents 'all items by some taste dimensions' and 'all users by some taste dimensions'. These dimensions are called ***latent or hidden features*** and we learn them from our data" ([Medium article: "ALS Implicit Collaborative Filtering"](https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe)).

For collaborative ALS, we will want our data to be shaped something like below, where the column numbers represent Recipe ID:                                            
 
User    | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 |...| n  
:-------|:--|---|---|---|---|---|---|---|---|---|---|---|---|---|---:  
user_01 | 0 | 0 | 5 | 4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 5 | 0 | 0 | 0  
user_02 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  
user_03 | 5 | 0 | 0 | 0 | 5 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 5 | 0 | 0  
user_04 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  
user_05 | 0 | 0 | 0 | 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 5 | 0 | 0 | 0  
user_06 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  
user_07 | 0 | 0 | 5 | 0 | 0 | 0 | 5 | 0 | 0 | 0 | 5 | 0 | 0 | 0 | 0  
  ...   |  |  |  |  |  |  |  | ... |  |  |  |  |  |  | ...  
user_n  | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 5 | 0 | 0 | 0  


# Model-based Rec. System

First we will split our RAW Interactions data in to a training dataset and testing dataset.

In [123]:
trainset, testset = train_test_split(idf, test_size = 0.2, random_state = 1984)

AttributeError: 'DataFrame' object has no attribute 'raw_ratings'

# Evaluation

# Reference

Reference:  
>(*Both personalized and content-based recommendation systems*) make use of different similarity metrics to determine how "similar" items are to one another. The most common similarity metrics are [**Euclidean distance**](https://en.wikipedia.org/wiki/Euclidean_distance), [**cosine similarity**](https://en.wikipedia.org/wiki/Cosine_similarity), [**Pearson correlation**](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) and the [**Jaccard index**](https://en.wikipedia.org/wiki/Jaccard_index) (useful with binary data). Each one of these distance metrics has its advantages and disadvantages depending on the type of ratings you are using and the characteristics of your data.$_1$  
  

## Further Reading

For those of you visiting this page who are interested in reading more about recommender systems, below are fantastic resources that I have collected.  
$_n$$_o$$_w$ &nbsp;$_I$ &nbsp;$_a$$_m$ &nbsp;$_t$$_h$$_e$ &nbsp;$_r$$_e$$_c$$_o$$_m$$_m$$_e$$_n$$_d$$_a$$_t$$_i$$_o$$_n$ &nbsp;$_s$$_y$$_s$$_t$$_e$$_m$$_!$

[*Mining Massive Datasets: Chapter 9*](http://infolab.stanford.edu/~ullman/mmds/ch9.pdf), © Copyright Stanford University. Stanford, California 94305  
[Singular Value Decomposition (SVD) & Its Application In Recommender System](https://analyticsindiamag.com/singular-value-decomposition-svd-application-recommender-system/), Dr. Vaibhav Kumar

# Bibliography

1. [*Introduction to Recommender Systems* by Flatiron School](https://github.com/learn-co-curriculum/dsc-recommendation-system-introduction) is licensed under CC BY-NC-SA 4.0
>© 2018 Flatiron School, Inc.  
>© 2021 Flatiron School, LLC